### Ideas

- Create classes to represent Data, Metadata, and Dataset? (dataset being the combination of the two, with fns which link them; ie. for cross-checking the type defined in meta is what the data type actually is)
- For Meta, can be read in different formats, and then exported in different formats
  - For dataset, meta must be a particular format

- Filter out rows with all nulls
- Create unique_values function: input basename & output dataframe with unique values

In [ ]:
#| default_exp read

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
#| export
import pandas as pd
import polars as pl
import numpy as np
import pyreadstat
import pyspssio
from pathlib import Path
from typing import Optional, Any
from fastcore.utils import *

from pydantic import BaseModel
from pydantic.dataclasses import dataclass
# from pandera import ...

In [ ]:
data_dir = Path("../data/raw")
df, meta = pyspssio.read_sav(data_dir/"G214_PQ.sav")
df = pl.from_pandas(df)

In [ ]:
#|export
@dataclass
class Metadata:
    label: str
    field_values: dict[int, str]
    field_type: str
    field_width: int
    decimals: int
    variable_type: str

Test this works as expected

In [ ]:
Metadata(
    label = "Ever had back pain",
    field_values = {-99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

Metadata(label='Ever had back pain', field_values={-99: 'Missing', 0: 'No', 1: 'Yes'}, field_type='Numeric', field_width=3, decimals=0, variable_type='Nominal')

In [ ]:
#|export
def unpack_variable_types(input_dict):
    "..."
    field_type = {}
    decimals = {}
    
    for key, value in input_dict.items():
        if value.startswith('F'):
            field_type[key] = 'Numeric'
            dec = value.split('.')[1]
            decimals[key] = dec if dec != '0' else '0'
        elif value.startswith('DATE'):
            field_type[key] = 'Date'
            decimals[key] = '0'
        elif value.startswith('A'):
            field_type[key] = 'String'
            decimals[key] = '0'
    
    return field_type, decimals

def reformat_metadata(m: pyreadstat.metadata_container, # metadata from pyreadstat
                     ) -> dict[dict[str, Any]]:
    "Reformat metadata into a more readable and consistent format"
    field_type, decimals = unpack_variable_types(m.original_variable_types)
    metadata = {
        "Label": m.column_names_to_labels,
        "Field Type": field_type,
        "Field Width": m.variable_display_width,
        "Decimals": decimals,
        "Variable Type": m.variable_measure,
        "Field Values": m.variable_value_labels
    }
    return metadata

TODO: add Data and Metadata classes, and nest them in the Dataset class

In [ ]:
#| export
def read_pyreadstat(data_dir: str|Path, 
                    file: str, 
                    cols: Optional[list[str]] = None
                    ) -> pl.LazyFrame:
    df, meta = pyreadstat.read_sav(f"{data_dir}/{file}", usecols=cols)
    df = pl.from_pandas(df).lazy()
    return df, meta

In [ ]:
#|export
@dataclass
class Dataset:
    file: str
    data_dir: str | Path
    prefix: Optional[str] = None
    variables: Optional[list[str]] = None

    def load_data(self) -> tuple[pl.LazyFrame, dict[dict[str, Any]]]:
        "Output data and metadata."
        df, meta = read_pyreadstat(self.data_dir, self.file, self.variables)
        meta = reformat_metadata(meta)
        return df, meta
    
        # def strip_prefix(self, df: pl.LazyFrame) -> pl.DataFrame:
    #     stripped_columns = {col: col.replace(self.prefix, "") for col in self.variables}
    #     df = df.rename(stripped_columns)
    #     return df

In [ ]:
df, meta = Dataset("G214_PQ.sav", data_dir).load_data()

In [ ]:
datasets = [
    Dataset("G214_PQ.sav", data_dir, "G214_PQ_", ["ID", "G214_PQ_PN17", "G214_PQ_PN25", "G214_PQ_PN34", "G214_PQ_PN35", "G214_PQ_PN36"]),
    Dataset("G214_SQ.sav", data_dir, "G214_SQ_", ["ID", "G214_SQ_PN17", "G214_SQ_PN25", "G214_SQ_PN34", "G214_SQ_PN35", "G214_SQ_PN36"]),
    Dataset("G217_PQ.sav", data_dir, "G217_PQ_", ["ID", "G217_PQ_PN17", "G217_PQ_PN25", "G217_PQ_PN34", "G217_PQ_PN35", "G217_PQ_PN36", "G217_PQ_PN38", "G217_PQ_PN9"]),
    Dataset("G217_SQ.sav", data_dir, "G217_SQ_", ["ID", "G217_SQ_PN17", "G217_SQ_PN25", "G217_SQ_PN34", "G217_SQ_PN35", "G217_SQ_PN36", "G217_SQ_PN38", "G217_SQ_PN9"])
]

In [ ]:
#|export
def read_and_filter_data(datasets: list[Dataset]
                         ) -> list[pl.LazyFrame]:
    """
    Take a list of `Dataset`s and return a list of the respective dataframes 
    and metadata for each dataset, filtered for the given columns.
    """
    dataframes = []
    metadata = []
    for ds in datasets:
        df, meta = ds.load_data()
        dataframes.append(df)
        metadata.append(meta)
    return dataframes, metadata

def combine_dataframes(dataframes: list[pl.LazyFrame]
                       ) -> pl.LazyFrame:
    "Take a list of dataframes and return a single, combined dataframe."
    combined_df = dataframes[0]
    for df in dataframes[1:]:
        combined_df = combined_df.join(df, on="ID", how="full", coalesce=True)
    return combined_df

In [ ]:
dataframes, metadata = read_and_filter_data(datasets)
df = combine_dataframes(dataframes)

In [ ]:
#| export
from collections import defaultdict

In [ ]:
#| export
def merge_dictionaries(dicts: list[dict[str, Any]]
                       ) -> dict[str, Any]:
    "Merge a series of nested dictionaries."
    merged_dict = defaultdict(dict)
    for d in dicts:
        for key, nested_dict in d.items():
            for nested_key, value in nested_dict.items():
                if nested_key not in merged_dict[key]:
                    merged_dict[key][nested_key] = value
                elif isinstance(value, dict):
                    merged_dict[key][nested_key].update(value)
    return dict(merged_dict)

In [ ]:
meta_merged = merge_dictionaries(metadata)

In [ ]:
pd.DataFrame(meta_merged).T.filter(regex="PN17")

,G214_PQ_PN17,G214_SQ_PN17,G217_PQ_PN17,G217_SQ_PN17
Label,Ever had back pain,Ever had back pain,Ever had back pain,Ever had back pain?
Field Type,Numeric,Numeric,Numeric,Numeric
Field Width,8,8,8,8
Decimals,0,0,0,0
Variable Type,scale,scale,scale,scale
Field Values,"{0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable',...","{0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable',...","{0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in inco...","{0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}"


In [ ]:
#| hide
nbdev_export()